# **Process Validation of Proposed Solution**

This notebook will implement the proposed solution as outlined in [Proposed Solution](https://www.notion.so/Proposed-Solution-d4f98653fc1d42019d89c5d0f66a5cc1). For any additional questions or suggestions, please either make a PR, or email Henry Gilbert directly at henry.gilbert@vanderbilt.edu.

---

## Setting Up Your Environment



Installing required libraries

In [ ]:
# Install requirements.txt
% pip install -r requirements.txt

Changing Directory to Parent to allow importing external data files. Please change the specified path based on where this repo exists for you locally.

In [ ]:
import os 

user_path = '~/GitHub/matcher'  # CHANGE THIS LINE AS NEEDED FOR YOUR ENVIRONMENT
os.chdir(os.path.expanduser(user_path))

--- 
## Importing Required Libraries

In [ ]:
from dataclasses import dataclass
from typing import List, Tuple
import numpy as np
import os
import glob
import scipy.stats as st
from scipy import stats
import dataclasses
import sys

---

## Defining Helper Classes

This outlines dataclasses that outline clear encapsulation of responsibility.

In [ ]:
@dataclass
class NormalDistribution:
    mean: float
    std: float
    
@dataclass
class Part:
    
    type: str
    sub_part_name: str
    sensor: str
    signals: List   # Signal is numpy array of (500,3) with [frequency, Z, X]

---

## Helper Functions

The following functions make up the core functionality of process. They will be individually introduced, explained and linked to the document before being used to demonstrate the process as a whole.

### *Loading Part Data*

As outlined in the [Individual Part PDF Estimation](https://www.notion.so/Proposed-Solution-d4f98653fc1d42019d89c5d0f66a5cc1#5b7c3ecbeaac40c8a7a1e2097c50d364), a part will repeatably have it's signal measured until the Confidence Interval at a certain confidence percentage converges. The specific propsed alogorithim to implement is outlined [here](https://www.notion.so/Proposed-Solution-d4f98653fc1d42019d89c5d0f66a5cc1#221e9d7bf8094d9bb51a6f63b8958f5f). However, as this is merely a notebook demonstrating the solution, continual sampling of the part signal is not possible. Rather, we just load the part data from a local store and pretend that the measurements we have represent the converged solution.

In [ ]:
def load_part_data(part_type: str) -> List[Part]:
    """ Given a part type as a string, load the data from the data directory.
    For each part type, we create a new Part object that holds mutliple measurement.
    A given part will have multiple measurements. Each part type will have multiple parts.
    
    Args:
        part_type: The type of part to load. This is a string that corresponds to a directory
        
    Returns:
        A list of Part objects. """
    
    parts = []
    for part_dir in os.listdir(f'psig_matcher/data/{part_type}'):
        
        sensor = part_dir[1:]
        measurement_files = glob.glob(f'psig_matcher/data/{part_type}/{part_dir}/*.npy')
        measurements = [np.load(f) for f in measurement_files]
        parts.append(Part(part_type, part_dir, sensor, measurements))
    
    return parts

### *Limiting Dimensionality*

This is not explicitly outlined in the [Proposed Solution](https://www.notion.so/Proposed-Solution-d4f98653fc1d42019d89c5d0f66a5cc1); however, it is useful for the purpose of demonstration. Each part signal contains 500 points and is transformed into a multi-variant Gaussian distribution with 500 dimensions. The entrophy and information density of using all 500 points is so high that when using our limited dataset, we're unable to simulate any collisions at all. 

While perhaps an useful indicator for high security, it does not illustrate the relationship between altering the input Confidence Interval and Confidence Bound. Thus, we introduce to ability to limit the dimensionality of a given signal by only taking the first *X* points. This reduces the signals entrophy and allows for actual overlaps between the part signals that are useful for demonstrating collision analysis. 

In [ ]:
def limit_deminsionality(parts: List[Part], frequeny_indexes: List[int]) -> List[Part]:
    """Use only a subset of the frequencies for the analysis. This effectivley transforms the 
    500 dimension multivariant distribution to a n-dimentional distribution where n is the
    length of the frequency_indexes.
    
    Further, this assumes use of the X axis and drops both the Y and Z axis.
    
    Args:
        parts: A list of Part objects.
        frequency_indexes: A list of indexes to use for the analysis.
        
    Returns:
        A list of Part objects with the signals limited to the frequency_indexes. """
    
    return [
        dataclasses.replace(part, signals=[[signal[index][1] for index in frequeny_indexes] for signal in part.signals])
        for part in parts]

### *Computing Confidence Intervals*

This is a simple function that allows one to compute the Confidence Interval for a given list of values. If less than 30 values are given, then by common practice, a Student-T distribution is assumed and used to calculate the interval. Conversly, when given over 30 values, a normal distribution is assumed and used.

In [ ]:
def compute_normal_ci(x: List[float], confidence: float) -> Tuple[float, float]:
    """Computes the confidence interval for a given confidence bound. 
    
    Args:
        x: A list of floats.
        confidence: The confidence bound to use.
    
    Returns:
        A tuple of floats that represent the lower and upper bound of the confidence interval. """
        
    if len(x) < 30:
        return st.t.interval(confidence, len(x)-1, loc=np.mean(x), scale=st.sem(x))
    else:
        return stats.norm.interval(confidence, loc=np.mean(x), scale=np.std(x))

### *Estimating The Distribution*

As outlined in [Individual Part PDF Estimation](https://www.notion.so/Proposed-Solution-d4f98653fc1d42019d89c5d0f66a5cc1#5b7c3ecbeaac40c8a7a1e2097c50d364), we must estimate the actual PDF for a given part of a given part type. There is some level variance introduced into the measurement of a parts signal through a mutlitude of environmental factors. We can model this uncertainty by recoqnizing there does not exist one true signal for a given part. Rather, we represent all possible signals for a part as a distribution. We later use the part's signal distribution as a means to classify novel signals in [Part Classification](https://www.notion.so/Proposed-Solution-d4f98653fc1d42019d89c5d0f66a5cc1#80419e83719348fb8cfb940b2819c731)

As we do not have the entire set of all possible signals for each part, rather only a small subset, we must recoqnize that our calculated PDF is only an estiamte of the true PDF. To account for this uncertainty, we compute the Confidence Interval and use that to derive the standard deviation that is used in in the PDF formulation. Thus, our PDF should be a bit more general than what is represented in our sample data to represent our inherent Epistemic Uncertaintity. 

In [ ]:
def estimate_dist(x: List[float], confidence: float) -> NormalDistribution:
    """Estimate the normal distribution for the given data. If the data is less than 30,
    then we use the t-distribution. Otherwise, we use the normal distribution.
    
    This is done using: https://handbook-5-1.cochrane.org/chapter_7/7_7_3_2_obtaining_standard_deviations_from_standard_errors_and.htm#:~:text=The%20standard%20deviation%20for%20each,should%20be%20replaced%20by%205.15. """
    
    val_comp = st.t.ppf if len(x) < 30 else stats.norm.ppf
    lower, upper = compute_normal_ci(x, confidence)
   
    val = val_comp(confidence, len(x)-1)
    std = np.sqrt(len(x))*(upper-lower)*val
    return NormalDistribution(np.mean(x, axis=0), std)

### *Probability a Point Came From a Distribution*

As outlined in [Part Classification](https://www.notion.so/Proposed-Solution-d4f98653fc1d42019d89c5d0f66a5cc1#80419e83719348fb8cfb940b2819c731), we are using a part's PDF to classify a signal. Very broadly the idea here is given a novel signal, we compute the probability that it came it from each distribution in the batch of parts. Whatever probability is highest, the signal is classified as coming from that distribution and thus, that part.

Please note, the actual part classifiction is quite a bit more nuanced and takes into account a hyperparameter of confidence bounds. This will be fully explained later in this notebook, or is outlined [here](https://www.notion.so/Proposed-Solution-d4f98653fc1d42019d89c5d0f66a5cc1#c9b37e585e304fdcaf83fb21339aad2a)

In [ ]:
def probability_of_multivariant_point(mu: List[float], cov: List[List[float]], x: List[float]) -> float:
    """ Given a the mean and covariance of a multivariant distribution, compute the probability
    that a given point is in the distribution. 
    
    Please note this solution is derived from: 
    https://stats.stackexchange.com/questions/331283/how-to-calculate-the-probability-of-a-data-point-belonging-to-a-multivariate-nor
    
    and needs to be verified and independently proven. 
    
    Args:
        mu: The mean of the multivariant distribution.
        cov: The covariance of the multivariant distribution.
        x: The point to compute the probability of.
        
    Returns:
        The probability that the point is in the distribution. """
    
    m_dist_x = np.dot((x-mu).transpose(),np.linalg.inv(cov))
    m_dist_x = np.dot(m_dist_x, (x-mu))
    return 1-stats.chi2.cdf(m_dist_x, 3)

---
## The Actual Process

The below defined functions combine the previously defined helper functions to create a functioning example of the [Proposed Solution](https://www.notion.so/Proposed-Solution-d4f98653fc1d42019d89c5d0f66a5cc1)

### *Estimating Collision*

The full proposed methodology along with proofs is outlined in [Collision Estimation](https://www.notion.so/Proposed-Solution-d4f98653fc1d42019d89c5d0f66a5cc1#a75ac7d53e544761a2adf1a680cab12f). Broadly speaking, at this point we would have the a set of PDFs for each part representing the part signals sample space. We would like to know the collision of the system for two reasons, it gives the security and the classification accuracy. 

The collision is calculated using signals derived only from non-counterfit parts all of the same part type and all from the same manufacturing process. Thus, if assume that any deviation in the manufacturing process increases the variance the measured signal, this collision estimate forms an upper bound for the parts security. For example, if we compute a collision of 0.5%, that means we estimate that any manufactured parts from the same producer have a 0.5% chance of their signal not being unique enough to classify. As deviation from manufacturing process increases variance, the probability of collision for a counterfit would be <<< 0.5%. Additionally, as we are using the parts' PDFs to classify a new signal, the collision also serves as the estimated classification accuracy for a given batch. 

However, as each part's PDF is a multivariant normal distribution, we can not directly compute the overlap of the PDFs as this is an intractable problem. Thus, we will estimate the collision rate of the system using Monte Carlo Sampling. 

Specifically, the process starts by estimating the PDF for each part using the supplied *part_pdf_ci*. We would like to estimate the collision using sample signals that are representitive of what can be expected for that part type, rather than just what we've recorded. To acomplish this, a meta PDF of part type signals is created using all signals from all parts and the supplied *meta_pdf_ci*.

We then draw *X* samples dictated by the *samples* arguement. For each sampled signal, we compute the probability that it came from each Parts computed PDF. We then filter the list of probabilities by those that are under the supplied *confidence_bound*. As outlined in [Collision Estimation](https://www.notion.so/Proposed-Solution-d4f98653fc1d42019d89c5d0f66a5cc1#a75ac7d53e544761a2adf1a680cab12f), this hyperparameter allows the tuning of collision rate and false negative rate. As our minimum confidence shrinks, we accept more classifications, reducing the false negative rate, but increasing the collision rate as the acceptance range each part PDF grows, increasing the potential area of overlap. Conversly, raising the minimum confidence will increase the false negative rate, but decrease the collision rate as the acceptance range each part PDF shrinks. Thus, the producer and consumer can tweak these hyperparameters to achieve the desired collision and false negative rate. 

Once the probabilities have been filtered, we calculate how many sampled signals resulted in a acceptance classification for more than one part PDF. This is then used to compute the estimated collision rate.

In [ ]:
def estimate_overlap_of_set_with_sample_signals(parts: List[Part], samples: int, meta_pdf_ci: float, part_pdf_ci: float, confidence_bound: float) -> float:
    """ Given a set of parts, estimate the overlap of the set with the sample signals. 
    Sample signals are derived from a meta distribution of signals derived from all measurements. 
    Signal classification is contrained to the confidence bound.
    
    Args:
        parts: A list of Part objects.
        samples: The number of samples to use for the meta distribution.
        meta_pdf_ci: The confidence bound to use for the meta distribution.
        part_pdf_ci: The confidence bound to use for the part distribution.
        confidence_bound: The confidence bound to use for the signal classification.
        
    Returns:
        The estimated overlap of the set with the sample signals. """
    
    min_confidence = 1 - confidence_bound
    signals = [
        signal for part in parts 
        for signal in part.signals]
    
    part_pdfs = [estimate_dist(part.signals, part_pdf_ci) for part in parts]
    sample_pdf = estimate_dist(signals, meta_pdf_ci)
    state_space_samples = np.random.multivariate_normal(sample_pdf.mean, np.diag(sample_pdf.std), samples)
    
    sample_confidences = [
        [probability_of_multivariant_point(pdf.mean, np.diag(pdf.std), sample) for pdf in part_pdfs]
        for sample in state_space_samples]

    filtered_confidences = [
        list(filter(lambda confidence: confidence >= min_confidence, sample_confidence))
        for sample_confidence in sample_confidences]

    # We're ok with up to 1 match, but every one more than that is a conflict.
    collisions = [max(len(confidences)-1, 0) for confidences in filtered_confidences]
    return sum(collisions)/(samples*len(part_pdfs))

### *Meta Markov Analysis*

The title is perhaps a bit tongue and cheek. The below function is explicitly outlined [here](https://www.notion.so/Proposed-Solution-d4f98653fc1d42019d89c5d0f66a5cc1#1889b6ec53b440ee9bbbf5001dc87488). Broadly, it seeks to answer the question; how many samples do we need for our Monte Carlo Sampling? Naturally, there is no real analyitical solution for this and often depends on the data and problem domain. 

However, we are unsatified merely guessing at a 'good enough value' and would like to produce a concrete estimation for the system collision rate. To acomplish this, we use the same convergence algorithim as outlined [here](https://www.notion.so/Proposed-Solution-d4f98653fc1d42019d89c5d0f66a5cc1#221e9d7bf8094d9bb51a6f63b8958f5f). Specifically, we run the above defined *estimate_overlap_of_set_with_sample_signals*, storing the estimated collision rate each time. Using the growing history of collision rates, we compute and store the confidence interval for the set of collision rates. Once we've run the Monte Carlo Sampling at least 100 times, we begin to check for convergence. Specifically, if the average of the past 10 confidence ranges have not decreased in relation to the average of the past 100, we consider the estimate to have converged. 

The upper bound of the estimated collision rate is then returned.

In [ ]:
def run_meta_markov_multivariant_analysis(parts: List[Part], part_dim: int, num_samples: int, meta_pdf_ci: float, part_pdf_ci: float, confidence_bound: float):
    """ Runs the Monte Carlo Approximation of multivariant collision using the signal sample meta
    pdf methodoly. The Monte Carlo Approximation will continually be run until the confidence interval
    converges and the average of the previous 10 runs is not smaller than the average of the previous 100 runs. 
    
    Args:
        parts: A list of Part objects.
        part_dim: The dimension of the part.
        num_samples: The number of samples to use for the meta distribution.
        meta_pdf_ci: The confidence bound to use for the meta distribution.
        part_pdf_ci: The confidence bound to use for the part distribution.
        confidence_bound: The confidence bound to use for the signal classification.
        
    Returns:
        A upper bound on the estimated system collision rate. """
    
    collisions = []
    confidence_ranges = []
    while True:
        
        multivariant_parts = limit_deminsionality(parts, list(range(part_dim)))
        collision_rate = estimate_overlap_of_set_with_sample_signals(multivariant_parts, num_samples, meta_pdf_ci, part_pdf_ci, confidence_bound)
        
        collisions.append(collision_rate)
        lower, upper = compute_normal_ci(collisions, 0.95)
        confidence_ranges.append(upper - lower)
        
        if len(confidence_ranges) > 100 and np.mean(confidence_ranges[-10:]) >= np.mean(confidence_ranges[-100:]):
            return upper

--- 

## Experimentation

The below sections gives example scenarios to illustrate the working code and validate the proposed approach.

### *Base Line*

In [ ]:
# Let's first establish a base line result using the following values
part_type = 'CON'
part_dim = 5
num_samples = 100
meta_pdf_ci = 0.999
part_pdf_ci = 0.999
confidence_bound = 0.999

con_parts = load_part_data(part_type)
estimated_upper_collision_rate = run_meta_markov_multivariant_analysis(
    con_parts, part_dim, num_samples, meta_pdf_ci, part_pdf_ci, confidence_bound)
print(f"Upper collision rate: {estimated_upper_collision_rate * 100}%")



We should see an expected upper collision rate of around ~4.5%. This makes sense as our part dimensionality is severly limited to only 5 out of the possible 500. Let's experiment with how consistent our convergence algorithim is:

(This took about ~45's on a M1 Macbook Pro 16gb RAM)

In [ ]:
collision_rates = [
    run_meta_markov_multivariant_analysis(con_parts, part_dim, num_samples, meta_pdf_ci, part_pdf_ci, confidence_bound) 
    for _ in range(30)]

print(f"Average: {np.mean(collision_rates) * 100}%")
print(f"Standard Deviation: {np.std(collision_rates) * 100}%")

We should expect to see the average be around ~4.5% with a fairly low standard deviation. The variation in this process is caused by the random sampling in the Monte Carlo Simulation

### *Increased Confidence Intervals*

This section will run the same experiment, but with an increased *meta_pdf_ci* and increased *part_pdf_ci*. A lower confidence interval on the creation of the meta signal PDF and each part PDF should result in tighter bounds, less overlap and a lower collision rate. 

In [46]:
part_type = 'CON'
part_dim = 5
num_samples = 100
meta_pdf_ci = 0.99
part_pdf_ci = 0.99
confidence_bound = 0.999

estimated_upper_collision_rate = run_meta_markov_multivariant_analysis(
    con_parts, part_dim, num_samples, meta_pdf_ci, part_pdf_ci, confidence_bound)
print(f"Upper collision rate: {estimated_upper_collision_rate * 100}%")

Upper collision rate: 0.2598547692148609%


As expected, our estiamted collision rate dropped by a factor of ~20!

### *Decreased Confidence Bounds*

Decreasing the confidence bound raises the minimum probability for a signal to be classified as a coming from a PDF. Conceptually, this is similar to constraining the part PDFs and should also result in a lower estimated collision rate.

In [47]:
part_type = 'CON'
part_dim = 5
num_samples = 100
meta_pdf_ci = 0.999
part_pdf_ci = 0.999
confidence_bound = 0.99

estimated_upper_collision_rate = run_meta_markov_multivariant_analysis(
    con_parts, part_dim, num_samples, meta_pdf_ci, part_pdf_ci, confidence_bound)
print(f"Upper collision rate: {estimated_upper_collision_rate * 100}%")

Upper collision rate: 2.3698259507441963%


Once again, our hypothesis is validated and the estimated collision rate is decreased. 

### *Increasing Part Dimensionality*

Increasing the dimensions of the part signals should increase the entrophy of the derived PDFs and lower the estimated collision rate.

In [48]:
part_type = 'CON'
part_dim = 10
num_samples = 100
meta_pdf_ci = 0.999
part_pdf_ci = 0.999
confidence_bound = 0.999

estimated_upper_collision_rate = run_meta_markov_multivariant_analysis(
    con_parts, part_dim, num_samples, meta_pdf_ci, part_pdf_ci, confidence_bound)
print(f"Upper collision rate: {estimated_upper_collision_rate * 100}%")

Upper collision rate: 0.3310271637686424%


Our hypothesis is validated and the estimated collision rate was decreased by a factor of ~20. 

---

## Conclusion

Hopefully this notebook demonstrated the proposed solution in a digestable way and serves as a useful reference for the more abstract ideas outlined in the solution. Again, for any questions please reach out to henry.gilbert@vanderbilt.edu.